### Installing Dependencies

In [ ]:
%pip install "unstructured[all-docs]" unstructured-client watermark python-dotenv pydantic langchain langchain-community langchain_core langchain_openai chromadb

### Hardware Details

In [1]:
# Get installed versions
import watermark

%load_ext watermark
%watermark -n -v -m -g -b

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.26.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.5.0-45-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

Git hash: 

Git branch: 



fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


### Ignore Warnings

In [2]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Unstructured.io Lookup

In [3]:
import unstructured.partition

help(unstructured.partition)

Help on package unstructured.partition in unstructured:

NAME
    unstructured.partition

PACKAGE CONTENTS
    api
    auto
    common
    csv
    doc
    docx
    email
    epub
    html (package)
    image
    json
    lang
    md
    model_init
    msg
    odt
    org
    pdf
    pdf_image (package)
    ppt
    pptx
    rst
    rtf
    strategies
    text
    text_type
    tsv
    utils (package)
    xlsx
    xml

FILE
    /home/prince/Desktop/content/tutorials/financial_statement_bot/venv/lib/python3.10/site-packages/unstructured/partition/__init__.py




### Load PDF Document

In [11]:
from unstructured.partition.pdf import partition_pdf

pdf_path = "./data/Sample-Accounting-Income-Statement-PDF-File.pdf"

elements = partition_pdf(filename=pdf_path)

In [12]:
print(elements)

[<unstructured.documents.elements.Header object at 0x76d7b2f49db0>, <unstructured.documents.elements.NarrativeText object at 0x76d7b2f4af20>, <unstructured.documents.elements.NarrativeText object at 0x76d7b2f4af80>, <unstructured.documents.elements.NarrativeText object at 0x76d7b2f4b130>, <unstructured.documents.elements.Title object at 0x76d7b2f4b190>, <unstructured.documents.elements.NarrativeText object at 0x76d7b2f4b160>, <unstructured.documents.elements.Title object at 0x76d7b2f4b3d0>, <unstructured.documents.elements.NarrativeText object at 0x76d7b2f4b4c0>, <unstructured.documents.elements.Title object at 0x76d7b2f4b550>, <unstructured.documents.elements.NarrativeText object at 0x76d7b2f4b5e0>, <unstructured.documents.elements.Title object at 0x76d7b2f4b7f0>, <unstructured.documents.elements.NarrativeText object at 0x76d7b2f4b760>, <unstructured.documents.elements.Title object at 0x76d6eb73d510>, <unstructured.documents.elements.Title object at 0x76d6eb5ebdf0>, <unstructured.docu

In [14]:
print(f"Length of elements: {len(elements)}")

Length of elements: 257


In [15]:
import json

element_dict = [element.to_dict() for element in elements]
output = json.dumps(element_dict, indent=2)

In [16]:
print(output)

[
  {
    "type": "Header",
    "element_id": "a0abbacc53a8d1d05301a1e524b67251",
    "text": "Understanding Basic Financial Statements",
    "metadata": {
      "coordinates": {
        "points": [
          [
            184.2,
            42.56400000000008
          ],
          [
            184.2,
            54.56400000000008
          ],
          [
            427.6671,
            54.56400000000008
          ],
          [
            427.6671,
            42.56400000000008
          ]
        ],
        "system": "PixelSpace",
        "layout_width": 612,
        "layout_height": 792
      },
      "file_directory": "./data",
      "filename": "Sample-Accounting-Income-Statement-PDF-File.pdf",
      "languages": [
        "eng"
      ],
      "last_modified": "2024-08-13T12:09:27",
      "page_number": 1,
      "filetype": "application/pdf"
    }
  },
  {
    "type": "NarrativeText",
    "element_id": "6798bb9a66d4d15dbd0c6ae4c576a09c",
    "text": "During the accounting cycl

In [19]:
unique_element_type = set()

for element in element_dict:
    unique_element_type.add(element["type"])

In [20]:
print(unique_element_type)

{'Footer', 'ListItem', 'NarrativeText', 'Title', 'UncategorizedText', 'Header'}


### API Client

In [21]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [27]:
unstructured_api_key = os.getenv("UNSTRUCTURED_API_KEY")
unstructured_api_url = os.getenv("UNSTRUCTURED_API_URL")

In [28]:
from unstructured_client import UnstructuredClient

client = UnstructuredClient(
    api_key_auth=unstructured_api_key,
    server_url=unstructured_api_url
)

### Extract Table With Client API

In [24]:
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError
from unstructured.staging.base import dict_to_elements

In [29]:
with open(pdf_path, "rb") as f:
    files = shared.Files(
        content=f.read(),
        file_name=pdf_path
    )
    

req = shared.PartitionParameters(
    files=files,
    strategy="hi_res",
    hi_res_model_name="yolox",
    skip_infer_table_types=[],
    pdf_infer_table_structure=True
)

try:
    resp = client.general.partition(req)
    elements = dict_to_elements(resp.elements)
except SDKError as e:
    print(e)

INFO: Preparing to split document for partition.
INFO: Starting page number set to 1
INFO: Allow failed set to 0
INFO: Concurrency level set to 5
INFO: Splitting pages 1 to 7 (7 total)
INFO: Determined optimal split size of 2 pages.
INFO: Partitioning 3 files with 2 page(s) each.
INFO: Partitioning 1 file with 1 page(s).
INFO: Partitioning set #1 (pages 1-2).
INFO: Partitioning set #2 (pages 3-4).
INFO: Partitioning set #3 (pages 5-6).
INFO: Partitioning set #4 (pages 7-7).
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: Successfully partitioned set #1, elements added to the final result.
INFO: Successfully partitioned set #2, elements added to the final result.
INFO: Successfully partitioned set #3, elements added to the final result.
INFO: Successfully partiti

In [30]:
unique_element_type = set()


for element in elements:
    unique_element_type.add(element.category)

In [31]:
print(unique_element_type)

{'NarrativeText', 'Title', 'UncategorizedText', 'Table'}


In [32]:
tables = [element for element in elements if element.category == "Table"]

In [33]:
print(tables)

[<unstructured.documents.elements.Table object at 0x76d7b27142b0>, <unstructured.documents.elements.Table object at 0x76d7b27144c0>, <unstructured.documents.elements.Table object at 0x76d7b2714850>, <unstructured.documents.elements.Table object at 0x76d7b2714a60>, <unstructured.documents.elements.Table object at 0x76d7b2714af0>, <unstructured.documents.elements.Table object at 0x76d7b27156f0>, <unstructured.documents.elements.Table object at 0x76d7b27140d0>, <unstructured.documents.elements.Table object at 0x76d7b275f310>]


In [34]:
print(f"Length of tables: {len(tables)}")

Length of tables: 8


In [35]:
tables[0].text

"2002 ASSETS CURRENT Cash Accounts receivable Deposits and prepaid expenses Inventory $ 11,552 42,970 2,942 159,144 216,608 PROPERTY, PLANT AND EQUIPMENT (Note 2) 59,890 INVESTMENTS -- $ 276,498 LIABILITIES CURRENT Bank overdraft Bank loan Accounts payable and accrued liabilities Long-term debt - current portion Income tax payable $ -- -- 82,053 25,200 14,387 121,640 DUE TO SHAREHOLDER (Note 3) 51,591 LONG-TERM DEBT (Note 4) 86,100 259,331 SHAREHOLDER'S EQUITY STATED CAPITAL (Note 5) 1 RETAINED EARNINGS (DEFICIT) 17,166 17,167 $ 276,498 $ $ $ $ 2001 -- 50,595 2,688 156,657 209,940 76,318 45,001 331,259 9,474 60,000 91,343 -- -- 160,817 231,791 -- 392,608 1 (61,350) (61,349) 331,259"

In [36]:
tables[0].metadata

In [37]:
first_table_html = tables[0].metadata.text_as_html
print(first_table_html)

<table><thead><tr><th>ASSETS</th><th>2002</th><th>200</th></tr></thead><tbody><tr><td colspan="3">CURRENT</td></tr><tr><td>Cash</td><td>11,552 $</td><td>- $</td></tr><tr><td>Accounts receivable</td><td>42,970</td><td>50,595</td></tr><tr><td>Deposits and prepaid expenses</td><td>2,942</td><td>2,688</td></tr><tr><td>Inventory</td><td>159,144</td><td>156,657</td></tr><tr><td></td><td>216,608</td><td>209,940</td></tr><tr><td>PROPERTY, PLANT AND EQUIPMENT (Note 2)</td><td>59,890</td><td>76,318</td></tr><tr><td>INVESTMENTS</td><td>-</td><td>45,001</td></tr><tr><td></td><td>276,498
 $</td><td>331,259
 $</td></tr><tr><td colspan="3">LIABILITIES</td></tr><tr><td colspan="3">CURRENT</td></tr><tr><td>Bank overdraft</td><td>- - $</td><td>9,474 $</td></tr><tr><td>Bank loan</td><td>8-2-,053</td><td>60,000</td></tr><tr><td>Long-term - portion ncome tax payable</td><td>25,200 14,387</td><td>-</td></tr><tr><td></td><td>121,640</td><td>160,817</td></tr><tr><td>DUE TO SHAREHOLDER (Note 3)</td><td>51,591<

In [38]:
from io import StringIO
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True)
file_obj = StringIO(first_table_html)
tree = etree.parse(file_obj, parser)
print(etree.tostring(tree, pretty_print=True).decode())

<table>
  <thead>
    <tr>
      <th>ASSETS</th>
      <th>2002</th>
      <th>200</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td colspan="3">CURRENT</td>
    </tr>
    <tr>
      <td>Cash</td>
      <td>11,552 $</td>
      <td>- $</td>
    </tr>
    <tr>
      <td>Accounts receivable</td>
      <td>42,970</td>
      <td>50,595</td>
    </tr>
    <tr>
      <td>Deposits and prepaid expenses</td>
      <td>2,942</td>
      <td>2,688</td>
    </tr>
    <tr>
      <td>Inventory</td>
      <td>159,144</td>
      <td>156,657</td>
    </tr>
    <tr>
      <td/>
      <td>216,608</td>
      <td>209,940</td>
    </tr>
    <tr>
      <td>PROPERTY, PLANT AND EQUIPMENT (Note 2)</td>
      <td>59,890</td>
      <td>76,318</td>
    </tr>
    <tr>
      <td>INVESTMENTS</td>
      <td>-</td>
      <td>45,001</td>
    </tr>
    <tr>
      <td/>
      <td>276,498
 $</td>
      <td>331,259
 $</td>
    </tr>
    <tr>
      <td colspan="3">LIABILITIES</td>
    </tr>
    <tr>
      <td colspan="3">

In [39]:
from IPython.core.display import HTML

HTML(first_table_html)

In [40]:
import pandas as pd

dfs = pd.read_html(first_table_html)
df = dfs[0]
df.head()

,ASSETS,2002,200
0,CURRENT,CURRENT,CURRENT
1,Cash,"11,552 $",- $
2,Accounts receivable,42970,50595
3,Deposits and prepaid expenses,2942,2688
4,Inventory,159144,156657


## Examine Text Elements

In [43]:
texts = [el for el in elements if el.category != "Text"]
texts[1].text

'During the accounting cycle, the accounting system is used to track, organize and record the financial transactions of an organization. At the close of each period, the information is used to prepare the financial statements, which are usually composed of a balance sheet (statement of financial position); income statement (statement of income and expenses); statement of retained earnings (owners’ equity); and a statement of cash flow.'

In [44]:
extracted_text = ""

for cat in elements:
    if cat.category == "Formula":
        extracted_text += cat.text + "\n"
    if cat.category == "FigureCaption":
        extracted_text += cat.text + "\n"
    if cat.category == "NarrativeText":
        extracted_text += cat.text + "\n"
    if cat.category == "ListItem":
        extracted_text += cat.text + "\n"
    if cat.category == "Title":
        extracted_text += cat.text + "\n"
    if cat.category == "Address":
        extracted_text += cat.text + "\n"
    if cat.category == "EmailAddress":
        extracted_text += cat.text + "\n"
    if cat.category == "Table":
        extracted_text += cat.metadata.text_as_html + "\n"
    if cat.category == "Header":
        extracted_text += cat.text + "\n"
    if cat.category == "Footer":
        extracted_text += cat.text + "\n"
    if cat.category == "CodeSnippet":
        extracted_text += cat.text + "\n"
    if cat.category == "UncategorizedText":
        extracted_text += cat.text + "\n"

In [45]:
print(extracted_text)

Understanding Basic Financial Statements
During the accounting cycle, the accounting system is used to track, organize and record the financial transactions of an organization. At the close of each period, the information is used to prepare the financial statements, which are usually composed of a balance sheet (statement of financial position); income statement (statement of income and expenses); statement of retained earnings (owners’ equity); and a statement of cash flow.
A sample set of financial statements) is shown below.
Financial statements prepared by a Chartered Accountant with a Review Engagement Report or Audit Opinion attached, are prepared (unless noted otherwise) according to "Canadian generally accepted accounting principles", or GAAP. Financial statements that are only compiled or that have a "Notice to Reader" attached, are not necessarily prepared according to GAAP.
Balance Sheet
The balance sheet is based on the equation: assets = liabilities + owners' equity. It in

In [46]:
from IPython.core.display import Markdown

Markdown(extracted_text)

Understanding Basic Financial Statements
During the accounting cycle, the accounting system is used to track, organize and record the financial transactions of an organization. At the close of each period, the information is used to prepare the financial statements, which are usually composed of a balance sheet (statement of financial position); income statement (statement of income and expenses); statement of retained earnings (owners’ equity); and a statement of cash flow.
A sample set of financial statements) is shown below.
Financial statements prepared by a Chartered Accountant with a Review Engagement Report or Audit Opinion attached, are prepared (unless noted otherwise) according to "Canadian generally accepted accounting principles", or GAAP. Financial statements that are only compiled or that have a "Notice to Reader" attached, are not necessarily prepared according to GAAP.
Balance Sheet
The balance sheet is based on the equation: assets = liabilities + owners' equity. It indicates everything the company owns (assets), everything the company owes to creditors (liabilities) and the value of the ownership stake in the company (shareholders' equity, or capital). The balance sheet date is the ending date of the period or year, and is a continuation of the amounts recorded since the inception of the company or organization. The balance sheet is a "snapshot" of the financial position of the company at the balance sheet date and shows the accumulated balance of the accounts. Assets and liabilities are separated between current and long-term, where current items are those items, which will be realized or paid, within one year of the balance sheet date. Typical current assets are cash, prepaid expenses, accounts receivable and inventory.
Income Statement
An income statement is a type of summary flow report that lists and categorizes the various revenues and expenses that result from operations during a given period - a year, a quarter or a month. The difference between revenues and expenses represents a company's net income or net loss. The amounts shown in the income statement are the amounts recorded for the given period - a year, a quarter or a month. The next period’s income statement will start over with all amounts reset to zero. While the balance sheet shows accumulated balances since inception, the income statement only shows the amounts earned or expensed during the period in question.
Statement of Retained Earnings
The statement of retained earnings shows the amount of accumulated earnings that have been retained within the company since its inception. At the end of each fiscal year-end, the amount of net income or loss is added to the opening amount of retained earnings to arrive at the closing retained earnings. Retained earnings can be decreased by such items as dividends paid to shareholders. On the sample financial statements shown below, the statement of retained earnings is combined with the income statement presentation.
Statement of Cash Flow
The statement of cash flow shows all sources and uses of a company's cash during the accounting period. Sources of cash listed on the statement include revenues, long-term financing, sales of non- current assets, an increase in any current liability account or a decrease in any current asset account. Uses of cash include operating losses, debt repayment, equipment purchases and increases in current asset accounts.
XYZ COMPANY LIMITED BALANCE SHEET AS AT JUNE 30, 2002
UNAUDITED - See "Notice to Reader"
<table><thead><tr><th>ASSETS</th><th>2002</th><th>200</th></tr></thead><tbody><tr><td colspan="3">CURRENT</td></tr><tr><td>Cash</td><td>11,552 $</td><td>- $</td></tr><tr><td>Accounts receivable</td><td>42,970</td><td>50,595</td></tr><tr><td>Deposits and prepaid expenses</td><td>2,942</td><td>2,688</td></tr><tr><td>Inventory</td><td>159,144</td><td>156,657</td></tr><tr><td></td><td>216,608</td><td>209,940</td></tr><tr><td>PROPERTY, PLANT AND EQUIPMENT (Note 2)</td><td>59,890</td><td>76,318</td></tr><tr><td>INVESTMENTS</td><td>-</td><td>45,001</td></tr><tr><td></td><td>276,498
 $</td><td>331,259
 $</td></tr><tr><td colspan="3">LIABILITIES</td></tr><tr><td colspan="3">CURRENT</td></tr><tr><td>Bank overdraft</td><td>- - $</td><td>9,474 $</td></tr><tr><td>Bank loan</td><td>8-2-,053</td><td>60,000</td></tr><tr><td>Long-term - portion ncome tax payable</td><td>25,200 14,387</td><td>-</td></tr><tr><td></td><td>121,640</td><td>160,817</td></tr><tr><td>DUE TO SHAREHOLDER (Note 3)</td><td>51,591</td><td>231,791</td></tr><tr><td>LONG-TERM DEBT (Note 4)</td><td>86,100</td><td>-</td></tr><tr><td></td><td>259,331</td><td>392,608</td></tr><tr><td colspan="3">SHAREHOLDER'S EQUITY</td></tr><tr><td>STATED CAPITAL (Note 5)</td><td>1</td><td>1</td></tr><tr><td>RETAINED EARNINGS (DEFICIT)</td><td>17,166</td><td>(61,350)</td></tr><tr><td></td><td>17,167</td><td>(61,349)</td></tr><tr><td></td><td>276,498
 $</td><td>331,259
 $</td></tr></tbody></table>
APPROVED
The accompanying summary of significant accounting policies and notes are an integral part of these financial statements.
XYZ COMPANY LIMITED STATEMENT OF INCOME AND RETAINED EARNINGS FOR THE YEAR ENDED JUNE 30, 2002
UNAUDITED - See "Notice to Reader"
<table><thead><tr><th></th><th>== 2002</th><th>AT 2001</th></tr></thead><tbody><tr><td>REVENUE</td><td>$ 1,104,786</td><td>$ 1,133,736</td></tr><tr><td colspan="3">COST OF SALES</td></tr><tr><td>Opening inventory</td><td>156,657</td><td>146,278</td></tr><tr><td>Delivery</td><td>1,607</td><td>1,249</td></tr><tr><td rowspan="2">Purchases</td><td>740,994</td><td>794,101</td></tr><tr><td>899,258</td><td>941,628</td></tr><tr><td rowspan="2">Closing inventory</td><td>159,144</td><td>156,657</td></tr><tr><td>740,114</td><td>784,971</td></tr><tr><td>GROSS PROFIT</td><td>364,672</td><td>348,765</td></tr><tr><td>OPERATING EXPENSES (schedule)</td><td>286,817</td><td>339,905</td></tr><tr><td>INCOME FROM OPERATIONS</td><td>77,855</td><td>8,860</td></tr><tr><td colspan="3">OTHER INCOME (EXPENSES)</td></tr><tr><td>Loss on disposal of property, plantand equipment</td><td></td><td>(387)</td></tr><tr><td>Gain on sale of investment</td><td>1_6_,149</td><td>- -</td></tr><tr><td>Miscellaneous</td><td>(1,101)</td><td>337</td></tr><tr><td></td><td>15,048</td><td>(50)</td></tr><tr><td>NET INCOME BEFORE TAX</td><td>92,903</td><td>8,810</td></tr><tr><td>INCOME TAX EXPENSE</td><td>14,387</td><td>--</td></tr><tr><td>NET INCOME</td><td>78,516</td><td>8,810</td></tr><tr><td>(DEFICIT) - Beginning of Year</td><td>(61,350)</td><td>(54,160)</td></tr><tr><td>DIVIDENDS</td><td>--</td><td>(16,000)</td></tr><tr><td>RETAINED EARNINGS (DEFICIT) - End of Year</td><td>$ 17,166</td><td>$ (61,350)</td></tr></tbody></table>
The accompanying summary of significant accounting policies and notes are an integral part of these financial statements.
XYZ COMPANY LIMITED STATEMENT OF CASH FLOW FOR THE YEAR ENDED JUNE 30, 2002
<table><thead><tr><th>STATEMENT OF CASH FLOW FOR THE YEAR ENDED J UNE 30, 2002 UNAUDITED - See "Notice to Reader"</th><th>2002</th><th>N —_</th></tr><tr><th></th><th></th><th></th></tr></thead><tbody><tr><td colspan="3">CASH FLOWS FROM OPERATING ACTIVITIES</td></tr><tr><td colspan="3">78,516
 8,810
 Net income for the year $
 $</td></tr><tr><td colspan="3">Adjustment for:</td></tr><tr><td colspan="3">Amortization 17,854 16,856</td></tr><tr><td>Loss on disposal of property, plant and equipment</td><td></td><td>387</td></tr><tr><td>Gain on disposal of investment</td><td>(1-6;,149)</td><td>- -</td></tr><tr><td>Cash derived from operations</td><td>80,221</td><td>26,053</td></tr><tr><td colspan="3">Decrease (increase) in working capital items</td></tr><tr><td colspan="3">Accounts receivable 7,625 23,380</td></tr><tr><td>and</td><td>(254)</td><td>688</td></tr><tr><td>Deposits prepaid expenses</td><td></td><td></td></tr><tr><td>Inventory Accounts and accrued liabilities</td><td>(2,487) (9,290)</td><td>(904) 34,543</td></tr><tr><td>payable debt current</td><td>25,200</td><td></td></tr><tr><td>Long-term - portion Income tax</td><td></td><td>_2_,206</td></tr><tr><td>payable Cash flows from operating activities</td><td>14,387 115,402</td><td>85,966</td></tr><tr><td colspan="3" rowspan="2">CASH FLOWS FROM INVESTING ACTIVITIES Acquisition of property, plant and equipment (1,426) (10,342</td></tr><tr><td>Proceeds from disposal of property, plant and equipment</td><td></td><td>3,113</td></tr><tr><td></td><td>6_1-,150</td><td></td></tr><tr><td>Proceeds from disposal of investment</td><td></td><td></td></tr><tr><td>Dividends</td><td>-</td><td>(1;;,000</td></tr><tr><td></td><td></td><td></td></tr><tr><td>Cash flows from investing activities</td><td>59,724</td><td>(23,229)</td></tr><tr><td>CASH FLOWS FROM FINANCING ACTIVITIES</td><td></td><td></td></tr><tr><td>Advances from (repayments to) shareholder</td><td>(180,200)</td><td>(15({,000</td></tr><tr><td>Acquisition of (repayment of) long-term debt</td><td>86,100</td><td>-</td></tr><tr><td colspan="3">(94,100)
 (150,000)</td></tr><tr><td>NET INCREASE (DECREASE) IN CASH RESOURCES</td><td>81,026</td><td>(87,263)</td></tr><tr><td>CASH (DEFICIENCY) RESOURCES - Beginning of Y ear</td><td>(69,474)</td><td>17,789</td></tr><tr><td>CASH RESOURCES (DEFICIENCY) - End of Year</td><td>11,552
 $</td><td>(69,474)
 $</td></tr><tr><td colspan="3">Cash resources (deficiency) is comprised of:</td></tr><tr><td>Cash</td><td>11,552 $</td><td>$</td></tr><tr><td>Bank overdraft</td><td>-</td><td>(_5;,474</td></tr><tr><td>Bank loan</td><td>-</td><td>(60,000</td></tr><tr><td></td><td>$ 11,552</td><td>(69,474)
 $</td></tr></tbody></table>
The accompanying summary of significant accounting policies and notes are an integral part of these financial statements.
XYZ COMPANY LIMITED SCHEDULE OF OPERATING EXPENSES FOR THE YEAR ENDED JUNE 30, 2002
UNAUDITED - See "Notice to Reader"
2002
2001
<table><tbody><tr><td>Advertising</td><td>18,801 $</td><td>30,715 $</td></tr><tr><td>Amortization</td><td>17,854</td><td>16,856</td></tr><tr><td>Bad debts</td><td>199</td><td>179</td></tr><tr><td>Bank charges and interest</td><td>5,510</td><td>9,356</td></tr><tr><td>Insurance</td><td>4,779</td><td>4,035</td></tr><tr><td>Interest on long-term debt</td><td>11,876</td><td></td></tr><tr><td>Legal and accounting</td><td>13,155</td><td>1-1-,766</td></tr><tr><td>Management fees</td><td></td><td>12,525</td></tr><tr><td>Memberships and licences</td><td>-.';,212</td><td>3,413</td></tr><tr><td>Office and general</td><td>12,382</td><td>15,155</td></tr><tr><td>Rent and utilities</td><td>18,795</td><td>21,955</td></tr><tr><td>Repairs and maintenance</td><td>4,289</td><td>4,979</td></tr><tr><td>Subcontracting</td><td>19,825</td><td>18,692</td></tr><tr><td>Supplies</td><td>4,571</td><td>8,557</td></tr><tr><td>Travel</td><td>3,736</td><td>8,003</td></tr><tr><td>Wages and benefits</td><td>142,713</td><td>168,806</td></tr><tr><td>Vehicle</td><td>5,120</td><td>4,913</td></tr><tr><td></td><td>286,817
 $</td><td>$ 339,905</td></tr></tbody></table>
The accompanying summary of significant accounting policies and notes are an integral part of these financial statements.
XYZ COMPANY LIMITED NOTES TO THE FINANCIAL STATEMENTS FOR THE YEAR ENDED JUNE 30, 2002
UNAUDITED - See "Notice to Reader"
1. SIGNIFICANT ACCOUNTING POLICIES AND GENERAL INFORMATION
Nature of Business
The company is a Canadian-controlled private corporation subject to the Business Corporations Act, 1982 (Ontario), was incorporated in May 1995 and operates as a manufacturer of widgets in Anytown, Ontario.
Significant Accounting Policies
INVENTORY The inventory is valued at the lower of cost or market, with cost being determined on a first-in, first-out basis.
PROPERTY, PLANT AND EQUIPMENT Property, plant and equipment are stated at cost less accumulated amortization. Amortization is recorded at rates designed to amortize the cost of capital assets over their estimated useful lives. Amortization rates used are as follows:
<table><tbody><tr><td>Furniture and equipment</td><td>20% declining balance</td></tr><tr><td>Vehicle</td><td>30% declining balance</td></tr><tr><td>Computer equipment</td><td>30% declining balance</td></tr><tr><td>Leasehold improvements</td><td>straight-line 5 years</td></tr></tbody></table>
PROPERTY, PLANT AND EQUIPMENT
Net Book Value 2002
<table><thead><tr><th></th><th>Cost</th><th>Accumulated Amortization</th><th>Book Value 2002</th><th>Book Value 2001</th></tr></thead><tbody><tr><td>Furniture and equipment</td><td>21,500 $</td><td>$ 11,300</td><td>10,200 $</td><td>$ 12,750</td></tr><tr><td>Vehicle</td><td>26,486</td><td>15,460</td><td>11,026</td><td>15,752</td></tr><tr><td>Computer equipment</td><td>22,210</td><td>6,457</td><td>15,753</td><td>20,466</td></tr><tr><td>Leasehold improvements</td><td>37,350</td><td>14,439</td><td>22,911</td><td>27,350</td></tr><tr><td></td><td>47,986
 $</td><td>26,760
 $</td><td>59,890
 $</td><td>$ 76,318</td></tr></tbody></table>
Net Book Value 2001
XYZ COMPANY LIMITED NOTES TO THE FINANCIAL STATEMENTS FOR THE YEAR ENDED JUNE 30, 2002
UNAUDITED - See "Notice to Reader"
3. DUE TO SHAREHOLDER
The amount due to the shareholder bears interest at a rate determined annually and has no fixed terms of repayment. Interest paid for 2002 was $1,823 (2001 - $6,831)
4. LONG - TERM DEBT
Bank term loan bearing interest at prime plus 2%, repayable in monthly principal instalments of $2,100.00 plus interest to November 2007, secured by a general security agreement on the assets of the company and a personal guarantee from the shareholder.
2002
$ 111,300
$
2001
--
<table><tbody><tr><td colspan="3"></td></tr><tr><td>2004</td><td>25,200 $</td><td></td></tr><tr><td>2005</td><td>25,200</td><td></td></tr><tr><td>2006</td><td>25,200</td><td></td></tr><tr><td>2007</td><td>10,500</td><td></td></tr><tr><td></td><td>86,100
 $</td><td></td></tr></tbody></table>
5. STATED CAPITAL
Authorized:
Unlimited number of Common shares Unlimited number of non-cumulative, redeemable, voting, Class "A" Special shares
<table><thead><tr><th></th><th>2002</th><th>200</th></tr></thead><tbody><tr><td colspan="3">Issued:</td></tr><tr><td>1 Common shares</td><td>$</td><td>1 $
 1</td></tr></tbody></table>


## Preprocessing Extracted Tables And Text

In [47]:
from typing import Any
from pydantic import BaseModel

class Element(BaseModel):
    type: str
    page_content: Any

# Categorize by type
categorized_elements = []

for element in elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", page_content=str(element.metadata.text_as_html)))
    elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))
    elif "unstructured.documents.elements.ListItem" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))
    elif "unstructured.documents.elements.Title" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))
    elif "unstructured.documents.elements.Address" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))
    elif "unstructured.documents.elements.EmailAddress" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))
    elif "unstructured.documents.elements.Header" in str(type(element)):
        categorized_elements.append(Element(type="CodeSnippet", page_content=str(element)))
    elif "unstructured.documents.elements.CodeSnippet" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))
    elif "unstructured.documents.elements.UncategorizedText" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))

In [48]:
# Tables
table_elements = [element for element in categorized_elements if element.type == "table"]
print(len(table_elements))

# Text
text_elements = [element for element in categorized_elements if element.type == "text"]
print(len(text_elements))

8
44


In [49]:
for text_element in text_elements:
    print(text_element.page_content)

Understanding Basic Financial Statements
During the accounting cycle, the accounting system is used to track, organize and record the financial transactions of an organization. At the close of each period, the information is used to prepare the financial statements, which are usually composed of a balance sheet (statement of financial position); income statement (statement of income and expenses); statement of retained earnings (owners’ equity); and a statement of cash flow.
A sample set of financial statements) is shown below.
Financial statements prepared by a Chartered Accountant with a Review Engagement Report or Audit Opinion attached, are prepared (unless noted otherwise) according to "Canadian generally accepted accounting principles", or GAAP. Financial statements that are only compiled or that have a "Notice to Reader" attached, are not necessarily prepared according to GAAP.
Balance Sheet
The balance sheet is based on the equation: assets = liabilities + owners' equity. It in

In [52]:
all_table_html = ""

for table in table_elements:
    all_table_html += table.page_content + "</br></br>"
    print(table.page_content)

<table><thead><tr><th>ASSETS</th><th>2002</th><th>200</th></tr></thead><tbody><tr><td colspan="3">CURRENT</td></tr><tr><td>Cash</td><td>11,552 $</td><td>- $</td></tr><tr><td>Accounts receivable</td><td>42,970</td><td>50,595</td></tr><tr><td>Deposits and prepaid expenses</td><td>2,942</td><td>2,688</td></tr><tr><td>Inventory</td><td>159,144</td><td>156,657</td></tr><tr><td></td><td>216,608</td><td>209,940</td></tr><tr><td>PROPERTY, PLANT AND EQUIPMENT (Note 2)</td><td>59,890</td><td>76,318</td></tr><tr><td>INVESTMENTS</td><td>-</td><td>45,001</td></tr><tr><td></td><td>276,498
 $</td><td>331,259
 $</td></tr><tr><td colspan="3">LIABILITIES</td></tr><tr><td colspan="3">CURRENT</td></tr><tr><td>Bank overdraft</td><td>- - $</td><td>9,474 $</td></tr><tr><td>Bank loan</td><td>8-2-,053</td><td>60,000</td></tr><tr><td>Long-term - portion ncome tax payable</td><td>25,200 14,387</td><td>-</td></tr><tr><td></td><td>121,640</td><td>160,817</td></tr><tr><td>DUE TO SHAREHOLDER (Note 3)</td><td>51,591<

In [53]:
Markdown(all_table_html)

<table><thead><tr><th>ASSETS</th><th>2002</th><th>200</th></tr></thead><tbody><tr><td colspan="3">CURRENT</td></tr><tr><td>Cash</td><td>11,552 $</td><td>- $</td></tr><tr><td>Accounts receivable</td><td>42,970</td><td>50,595</td></tr><tr><td>Deposits and prepaid expenses</td><td>2,942</td><td>2,688</td></tr><tr><td>Inventory</td><td>159,144</td><td>156,657</td></tr><tr><td></td><td>216,608</td><td>209,940</td></tr><tr><td>PROPERTY, PLANT AND EQUIPMENT (Note 2)</td><td>59,890</td><td>76,318</td></tr><tr><td>INVESTMENTS</td><td>-</td><td>45,001</td></tr><tr><td></td><td>276,498
 $</td><td>331,259
 $</td></tr><tr><td colspan="3">LIABILITIES</td></tr><tr><td colspan="3">CURRENT</td></tr><tr><td>Bank overdraft</td><td>- - $</td><td>9,474 $</td></tr><tr><td>Bank loan</td><td>8-2-,053</td><td>60,000</td></tr><tr><td>Long-term - portion ncome tax payable</td><td>25,200 14,387</td><td>-</td></tr><tr><td></td><td>121,640</td><td>160,817</td></tr><tr><td>DUE TO SHAREHOLDER (Note 3)</td><td>51,591</td><td>231,791</td></tr><tr><td>LONG-TERM DEBT (Note 4)</td><td>86,100</td><td>-</td></tr><tr><td></td><td>259,331</td><td>392,608</td></tr><tr><td colspan="3">SHAREHOLDER'S EQUITY</td></tr><tr><td>STATED CAPITAL (Note 5)</td><td>1</td><td>1</td></tr><tr><td>RETAINED EARNINGS (DEFICIT)</td><td>17,166</td><td>(61,350)</td></tr><tr><td></td><td>17,167</td><td>(61,349)</td></tr><tr><td></td><td>276,498
 $</td><td>331,259
 $</td></tr></tbody></table></br></br><table><thead><tr><th></th><th>== 2002</th><th>AT 2001</th></tr></thead><tbody><tr><td>REVENUE</td><td>$ 1,104,786</td><td>$ 1,133,736</td></tr><tr><td colspan="3">COST OF SALES</td></tr><tr><td>Opening inventory</td><td>156,657</td><td>146,278</td></tr><tr><td>Delivery</td><td>1,607</td><td>1,249</td></tr><tr><td rowspan="2">Purchases</td><td>740,994</td><td>794,101</td></tr><tr><td>899,258</td><td>941,628</td></tr><tr><td rowspan="2">Closing inventory</td><td>159,144</td><td>156,657</td></tr><tr><td>740,114</td><td>784,971</td></tr><tr><td>GROSS PROFIT</td><td>364,672</td><td>348,765</td></tr><tr><td>OPERATING EXPENSES (schedule)</td><td>286,817</td><td>339,905</td></tr><tr><td>INCOME FROM OPERATIONS</td><td>77,855</td><td>8,860</td></tr><tr><td colspan="3">OTHER INCOME (EXPENSES)</td></tr><tr><td>Loss on disposal of property, plantand equipment</td><td></td><td>(387)</td></tr><tr><td>Gain on sale of investment</td><td>1_6_,149</td><td>- -</td></tr><tr><td>Miscellaneous</td><td>(1,101)</td><td>337</td></tr><tr><td></td><td>15,048</td><td>(50)</td></tr><tr><td>NET INCOME BEFORE TAX</td><td>92,903</td><td>8,810</td></tr><tr><td>INCOME TAX EXPENSE</td><td>14,387</td><td>--</td></tr><tr><td>NET INCOME</td><td>78,516</td><td>8,810</td></tr><tr><td>(DEFICIT) - Beginning of Year</td><td>(61,350)</td><td>(54,160)</td></tr><tr><td>DIVIDENDS</td><td>--</td><td>(16,000)</td></tr><tr><td>RETAINED EARNINGS (DEFICIT) - End of Year</td><td>$ 17,166</td><td>$ (61,350)</td></tr></tbody></table></br></br><table><thead><tr><th>STATEMENT OF CASH FLOW FOR THE YEAR ENDED J UNE 30, 2002 UNAUDITED - See "Notice to Reader"</th><th>2002</th><th>N —_</th></tr><tr><th></th><th></th><th></th></tr></thead><tbody><tr><td colspan="3">CASH FLOWS FROM OPERATING ACTIVITIES</td></tr><tr><td colspan="3">78,516
 8,810
 Net income for the year $
 $</td></tr><tr><td colspan="3">Adjustment for:</td></tr><tr><td colspan="3">Amortization 17,854 16,856</td></tr><tr><td>Loss on disposal of property, plant and equipment</td><td></td><td>387</td></tr><tr><td>Gain on disposal of investment</td><td>(1-6;,149)</td><td>- -</td></tr><tr><td>Cash derived from operations</td><td>80,221</td><td>26,053</td></tr><tr><td colspan="3">Decrease (increase) in working capital items</td></tr><tr><td colspan="3">Accounts receivable 7,625 23,380</td></tr><tr><td>and</td><td>(254)</td><td>688</td></tr><tr><td>Deposits prepaid expenses</td><td></td><td></td></tr><tr><td>Inventory Accounts and accrued liabilities</td><td>(2,487) (9,290)</td><td>(904) 34,543</td></tr><tr><td>payable debt current</td><td>25,200</td><td></td></tr><tr><td>Long-term - portion Income tax</td><td></td><td>_2_,206</td></tr><tr><td>payable Cash flows from operating activities</td><td>14,387 115,402</td><td>85,966</td></tr><tr><td colspan="3" rowspan="2">CASH FLOWS FROM INVESTING ACTIVITIES Acquisition of property, plant and equipment (1,426) (10,342</td></tr><tr><td>Proceeds from disposal of property, plant and equipment</td><td></td><td>3,113</td></tr><tr><td></td><td>6_1-,150</td><td></td></tr><tr><td>Proceeds from disposal of investment</td><td></td><td></td></tr><tr><td>Dividends</td><td>-</td><td>(1;;,000</td></tr><tr><td></td><td></td><td></td></tr><tr><td>Cash flows from investing activities</td><td>59,724</td><td>(23,229)</td></tr><tr><td>CASH FLOWS FROM FINANCING ACTIVITIES</td><td></td><td></td></tr><tr><td>Advances from (repayments to) shareholder</td><td>(180,200)</td><td>(15({,000</td></tr><tr><td>Acquisition of (repayment of) long-term debt</td><td>86,100</td><td>-</td></tr><tr><td colspan="3">(94,100)
 (150,000)</td></tr><tr><td>NET INCREASE (DECREASE) IN CASH RESOURCES</td><td>81,026</td><td>(87,263)</td></tr><tr><td>CASH (DEFICIENCY) RESOURCES - Beginning of Y ear</td><td>(69,474)</td><td>17,789</td></tr><tr><td>CASH RESOURCES (DEFICIENCY) - End of Year</td><td>11,552
 $</td><td>(69,474)
 $</td></tr><tr><td colspan="3">Cash resources (deficiency) is comprised of:</td></tr><tr><td>Cash</td><td>11,552 $</td><td>$</td></tr><tr><td>Bank overdraft</td><td>-</td><td>(_5;,474</td></tr><tr><td>Bank loan</td><td>-</td><td>(60,000</td></tr><tr><td></td><td>$ 11,552</td><td>(69,474)
 $</td></tr></tbody></table></br></br><table><tbody><tr><td>Advertising</td><td>18,801 $</td><td>30,715 $</td></tr><tr><td>Amortization</td><td>17,854</td><td>16,856</td></tr><tr><td>Bad debts</td><td>199</td><td>179</td></tr><tr><td>Bank charges and interest</td><td>5,510</td><td>9,356</td></tr><tr><td>Insurance</td><td>4,779</td><td>4,035</td></tr><tr><td>Interest on long-term debt</td><td>11,876</td><td></td></tr><tr><td>Legal and accounting</td><td>13,155</td><td>1-1-,766</td></tr><tr><td>Management fees</td><td></td><td>12,525</td></tr><tr><td>Memberships and licences</td><td>-.';,212</td><td>3,413</td></tr><tr><td>Office and general</td><td>12,382</td><td>15,155</td></tr><tr><td>Rent and utilities</td><td>18,795</td><td>21,955</td></tr><tr><td>Repairs and maintenance</td><td>4,289</td><td>4,979</td></tr><tr><td>Subcontracting</td><td>19,825</td><td>18,692</td></tr><tr><td>Supplies</td><td>4,571</td><td>8,557</td></tr><tr><td>Travel</td><td>3,736</td><td>8,003</td></tr><tr><td>Wages and benefits</td><td>142,713</td><td>168,806</td></tr><tr><td>Vehicle</td><td>5,120</td><td>4,913</td></tr><tr><td></td><td>286,817
 $</td><td>$ 339,905</td></tr></tbody></table></br></br><table><tbody><tr><td>Furniture and equipment</td><td>20% declining balance</td></tr><tr><td>Vehicle</td><td>30% declining balance</td></tr><tr><td>Computer equipment</td><td>30% declining balance</td></tr><tr><td>Leasehold improvements</td><td>straight-line 5 years</td></tr></tbody></table></br></br><table><thead><tr><th></th><th>Cost</th><th>Accumulated Amortization</th><th>Book Value 2002</th><th>Book Value 2001</th></tr></thead><tbody><tr><td>Furniture and equipment</td><td>21,500 $</td><td>$ 11,300</td><td>10,200 $</td><td>$ 12,750</td></tr><tr><td>Vehicle</td><td>26,486</td><td>15,460</td><td>11,026</td><td>15,752</td></tr><tr><td>Computer equipment</td><td>22,210</td><td>6,457</td><td>15,753</td><td>20,466</td></tr><tr><td>Leasehold improvements</td><td>37,350</td><td>14,439</td><td>22,911</td><td>27,350</td></tr><tr><td></td><td>47,986
 $</td><td>26,760
 $</td><td>59,890
 $</td><td>$ 76,318</td></tr></tbody></table></br></br><table><tbody><tr><td colspan="3"></td></tr><tr><td>2004</td><td>25,200 $</td><td></td></tr><tr><td>2005</td><td>25,200</td><td></td></tr><tr><td>2006</td><td>25,200</td><td></td></tr><tr><td>2007</td><td>10,500</td><td></td></tr><tr><td></td><td>86,100
 $</td><td></td></tr></tbody></table></br></br><table><thead><tr><th></th><th>2002</th><th>200</th></tr></thead><tbody><tr><td colspan="3">Issued:</td></tr><tr><td>1 Common shares</td><td>$</td><td>1 $
 1</td></tr></tbody></table></br></br>

## Creating Embeddings

In [54]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [55]:
# LCEL 
summary_chain = (
    {"doc": lambda x: x}
    | ChatPromptTemplate.from_template("Summarize the following tables or text given below:\n\n{doc}")
    | ChatOpenAI(max_retries=3)
    | StrOutputParser()
)

In [56]:
# Table summaries
tables_content = [i.page_content for i in table_elements]
teable_summaries = summary_chain.batch(tables_content, {"max_concurrency": 5})

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [57]:
teable_summaries[:2]

["The table shows the assets, liabilities, and shareholder's equity of a company for the years 2002 and 200. \n\nAssets for 2002:\n- Current: $216,608\n- Property, plant and equipment: $59,890\n- Investments: $-\n\nTotal assets for 2002: $276,498\n\nAssets for 200:\n- Current: $209,940\n- Property, plant and equipment: $76,318\n- Investments: $45,001\n\nTotal assets for 200: $331,259\n\nLiabilities for 2002:\n- Current: $121,640\n- Due to shareholder: $51,591\n- Long-term debt: $86,100\n\nTotal liabilities for 2002: $259,331\n\nLiabilities for 200:\n- Current: $160,817\n- Due to shareholder: $231,791\n- Long-term debt: -\n\nTotal liabilities for 200: $392,608\n\nShareholder's equity for 2002: $17,167\nShareholder's equity for 200: $331,259",
 'In 2002, the revenue was $1,104,786, with a cost of sales including opening inventory of $156,657, delivery costs of $1,607, purchases of $740,994, closing inventory of $159,144, resulting in a gross profit of $364,672. Operating expenses were $2

In [58]:
# Text summaries
text_content = [i.page_content for i in text_elements]
text_summaries = summary_chain.batch(text_content, {"max_concurrency": 5})

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/cha

In [59]:
text_summaries[:2]

["The text provides an overview of basic financial statements, including the balance sheet, income statement, and cash flow statement. It explains the purpose of each statement and how they are used to assess the financial health of a company. The text also discusses key financial ratios that can be calculated using information from these statements to analyze a company's performance.",
 'The accounting cycle involves tracking, organizing, and recording financial transactions using an accounting system. At the end of each period, this information is used to prepare financial statements, including a balance sheet, income statement, statement of retained earnings, and statement of cash flow.']

## Multivector Retrieval

In [60]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [61]:
# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="financials",
    embedding_function=OpenAIEmbeddings(),
    persist_directory="./chroma_data",
)

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

# Table summaries embeddings and storage
doc_ids = [str(uuid.uuid4()) for _ in table_elements]
summary_tables = [Document(page_content=s,metadata={id_key: doc_ids[i]}) for i, s in enumerate(teable_summaries)]

retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(doc_ids, table_elements)))

# Text summaries embeddings and storage
doc_ids = [str(uuid.uuid4()) for _ in text_elements]
summary_texts = [Document(page_content=s,metadata={id_key: doc_ids[i]}) for i, s in enumerate(text_summaries)]

retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, text_elements)))

/home/prince/Desktop/content/tutorials/financial_statement_bot/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/home/prince/Desktop/content/tutorials/financial_statement_bot/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(
INFO: Anonymized telemetry enabled. See       

## Information Retrieval

In [72]:
retrieved_first_response = retriever.invoke("Give me information about the revenue")

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [73]:
print(retrieved_first_response)

[Element(type='table', page_content='<table><thead><tr><th></th><th>== 2002</th><th>AT 2001</th></tr></thead><tbody><tr><td>REVENUE</td><td>$ 1,104,786</td><td>$ 1,133,736</td></tr><tr><td colspan="3">COST OF SALES</td></tr><tr><td>Opening inventory</td><td>156,657</td><td>146,278</td></tr><tr><td>Delivery</td><td>1,607</td><td>1,249</td></tr><tr><td rowspan="2">Purchases</td><td>740,994</td><td>794,101</td></tr><tr><td>899,258</td><td>941,628</td></tr><tr><td rowspan="2">Closing inventory</td><td>159,144</td><td>156,657</td></tr><tr><td>740,114</td><td>784,971</td></tr><tr><td>GROSS PROFIT</td><td>364,672</td><td>348,765</td></tr><tr><td>OPERATING EXPENSES (schedule)</td><td>286,817</td><td>339,905</td></tr><tr><td>INCOME FROM OPERATIONS</td><td>77,855</td><td>8,860</td></tr><tr><td colspan="3">OTHER INCOME (EXPENSES)</td></tr><tr><td>Loss on disposal of property, plantand equipment</td><td></td><td>(387)</td></tr><tr><td>Gain on sale of investment</td><td>1_6_,149</td><td>- -</td></t

In [74]:
retriever_first_response_page_content = retrieved_first_response[0].page_content
print(retriever_first_response_page_content)

<table><thead><tr><th></th><th>== 2002</th><th>AT 2001</th></tr></thead><tbody><tr><td>REVENUE</td><td>$ 1,104,786</td><td>$ 1,133,736</td></tr><tr><td colspan="3">COST OF SALES</td></tr><tr><td>Opening inventory</td><td>156,657</td><td>146,278</td></tr><tr><td>Delivery</td><td>1,607</td><td>1,249</td></tr><tr><td rowspan="2">Purchases</td><td>740,994</td><td>794,101</td></tr><tr><td>899,258</td><td>941,628</td></tr><tr><td rowspan="2">Closing inventory</td><td>159,144</td><td>156,657</td></tr><tr><td>740,114</td><td>784,971</td></tr><tr><td>GROSS PROFIT</td><td>364,672</td><td>348,765</td></tr><tr><td>OPERATING EXPENSES (schedule)</td><td>286,817</td><td>339,905</td></tr><tr><td>INCOME FROM OPERATIONS</td><td>77,855</td><td>8,860</td></tr><tr><td colspan="3">OTHER INCOME (EXPENSES)</td></tr><tr><td>Loss on disposal of property, plantand equipment</td><td></td><td>(387)</td></tr><tr><td>Gain on sale of investment</td><td>1_6_,149</td><td>- -</td></tr><tr><td>Miscellaneous</td><td>(1,10

In [75]:
Markdown(retriever_first_response_page_content)

<table><thead><tr><th></th><th>== 2002</th><th>AT 2001</th></tr></thead><tbody><tr><td>REVENUE</td><td>$ 1,104,786</td><td>$ 1,133,736</td></tr><tr><td colspan="3">COST OF SALES</td></tr><tr><td>Opening inventory</td><td>156,657</td><td>146,278</td></tr><tr><td>Delivery</td><td>1,607</td><td>1,249</td></tr><tr><td rowspan="2">Purchases</td><td>740,994</td><td>794,101</td></tr><tr><td>899,258</td><td>941,628</td></tr><tr><td rowspan="2">Closing inventory</td><td>159,144</td><td>156,657</td></tr><tr><td>740,114</td><td>784,971</td></tr><tr><td>GROSS PROFIT</td><td>364,672</td><td>348,765</td></tr><tr><td>OPERATING EXPENSES (schedule)</td><td>286,817</td><td>339,905</td></tr><tr><td>INCOME FROM OPERATIONS</td><td>77,855</td><td>8,860</td></tr><tr><td colspan="3">OTHER INCOME (EXPENSES)</td></tr><tr><td>Loss on disposal of property, plantand equipment</td><td></td><td>(387)</td></tr><tr><td>Gain on sale of investment</td><td>1_6_,149</td><td>- -</td></tr><tr><td>Miscellaneous</td><td>(1,101)</td><td>337</td></tr><tr><td></td><td>15,048</td><td>(50)</td></tr><tr><td>NET INCOME BEFORE TAX</td><td>92,903</td><td>8,810</td></tr><tr><td>INCOME TAX EXPENSE</td><td>14,387</td><td>--</td></tr><tr><td>NET INCOME</td><td>78,516</td><td>8,810</td></tr><tr><td>(DEFICIT) - Beginning of Year</td><td>(61,350)</td><td>(54,160)</td></tr><tr><td>DIVIDENDS</td><td>--</td><td>(16,000)</td></tr><tr><td>RETAINED EARNINGS (DEFICIT) - End of Year</td><td>$ 17,166</td><td>$ (61,350)</td></tr></tbody></table>

## Chatbot Interface

In [76]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough

# Prompt template
template = """Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(temperature=0,model="gpt-4")
#model = ChatOpenAI(temperature=0,model="gpt-3.5-turbo")

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [77]:
response = chain.invoke("What is the Leasehold improvements cost")
print(response)

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The cost of Leasehold improvements is 37,350$.


In [78]:
retrieved_first_response = retriever.invoke("What is the Leasehold improvements cost")

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [79]:
retriever_first_response_page_content = retrieved_first_response[0].page_content
print(retriever_first_response_page_content)

<table><thead><tr><th></th><th>Cost</th><th>Accumulated Amortization</th><th>Book Value 2002</th><th>Book Value 2001</th></tr></thead><tbody><tr><td>Furniture and equipment</td><td>21,500 $</td><td>$ 11,300</td><td>10,200 $</td><td>$ 12,750</td></tr><tr><td>Vehicle</td><td>26,486</td><td>15,460</td><td>11,026</td><td>15,752</td></tr><tr><td>Computer equipment</td><td>22,210</td><td>6,457</td><td>15,753</td><td>20,466</td></tr><tr><td>Leasehold improvements</td><td>37,350</td><td>14,439</td><td>22,911</td><td>27,350</td></tr><tr><td></td><td>47,986
 $</td><td>26,760
 $</td><td>59,890
 $</td><td>$ 76,318</td></tr></tbody></table>


In [80]:
Markdown(retriever_first_response_page_content)

<table><thead><tr><th></th><th>Cost</th><th>Accumulated Amortization</th><th>Book Value 2002</th><th>Book Value 2001</th></tr></thead><tbody><tr><td>Furniture and equipment</td><td>21,500 $</td><td>$ 11,300</td><td>10,200 $</td><td>$ 12,750</td></tr><tr><td>Vehicle</td><td>26,486</td><td>15,460</td><td>11,026</td><td>15,752</td></tr><tr><td>Computer equipment</td><td>22,210</td><td>6,457</td><td>15,753</td><td>20,466</td></tr><tr><td>Leasehold improvements</td><td>37,350</td><td>14,439</td><td>22,911</td><td>27,350</td></tr><tr><td></td><td>47,986
 $</td><td>26,760
 $</td><td>59,890
 $</td><td>$ 76,318</td></tr></tbody></table>